# 4 Qubit QITE
This notebook is a guide to implement QITE on 4 qubits

We will refer to Pauli matrices by their indices: $[I, X, Y, Z] \equiv [0, 1, 2, 3]$
For consistency in notation, we use the following mapping for 2-qubit operators, composed of a Pauli matrices acting on each qubit. This is the indexing used throughout the code.

In [1]:
# nqbits = 4
pauli_indices = [0, 1, 2, 3]
pauli_pairs = [(i, j, k, l) for i in pauli_indices for j in pauli_indices for k in pauli_indices for l in pauli_indices]
# print(pauli_pairs)
pauli_pair_dict = {pauli_pairs[i] : i for i in range(len(pauli_pairs))}
# print(pauli_pair_dict)

# # nqbits = 3
# pauli_indices = [0, 1, 2, 3]
# pauli_pairs = [(i, j, k) for i in pauli_indices for j in pauli_indices for k in pauli_indices]
# # print(pauli_pairs)
# pauli_pair_dict = {pauli_pairs[i] : i for i in range(len(pauli_pairs))}
# # print(pauli_pair_dict)

# # nqbits = 2
# pauli_indices = [0, 1, 2, 3]
# pauli_pairs = [(i, j) for i in pauli_indices for j in pauli_indices]
# print(pauli_pairs)
# pauli_pair_dict = {pauli_pairs[i] : i for i in range(len(pauli_pairs))}
# print(pauli_pair_dict)

## Measure the Pauli Expectations

In **measure** we define circuits to measure the expectation value of any Pauli string. Read more: https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements

In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ

In [3]:
backend = BasicAer.get_backend('qasm_simulator')

In [4]:
# Circuit to measure the expectation value of any Pauli string
# For 2-qubit Pauli measurements, see https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements
def measure(qc, ro, qbits, idx):
    '''
    Compute multi-qubit Pauli expectations by changing Pauli measurements to a 
    diagonal ({Z, I}^n) basis
    '''
    
    nqbits = len(qbits)
    idx_pair = pauli_pairs[idx]
    
    # I x I ....
    cond1 = [0] * nqbits
    # Z x I ....
    cond2 = [3] + [0] * (nqbits - 1)
    
    if idx_pair == tuple(cond1) or idx_pair == tuple(cond2):
        return
    
    else:    
        
        # list of qubits q to apply cnot gates to entangele with 0 qubit, qc.cx(q,0)
        cnot_list = qbits.copy()
        
        # swap flag... whether there is a swap between two qubits
        swap = -1
        for i in range(nqbits):
            # qubit index, for swapping
            q = i
            if swap != -1 and q == swap[1]:
                q = swap[0]
                swap = -1
            # I
            if idx_pair[i] == 0:
                cnot_list[i] = -1
                if i == 0:
                    # get first qubit acted on by Pauli matrix other than I
                    for j in range(1,len(idx_pair)):
                        if idx_pair[j] != 0:
                            qc.swap(qbits[i],qbits[j])
                            swap = (i, j)
                            cnot_list[j] = -1
                            break        
            # X
            elif idx_pair[i] == 1:
                qc.h(qbits[q])
            # Y
            elif idx_pair[i] == 2:
                qc.rx(np.pi/2,qbits[q])
            # Z
            elif idx_pair[i] == 3:
                pass
            else:
                raise ValueError
                
#         cnot_list[0] = -1
        cnot_list = cnot_list[1:]
        
        # Apply CNOT gates between all qubits acted on by Pauli matrix other than I (control) and 0 qubit (target)
        for q in cnot_list:
            if q != -1:
                qc.cx(q, qbits[0])

        # Measure the 0 qubit
        qc.measure(qbits[0],ro[0])
                

## Propagate the State

In **propagate**, we loop through the different values store in alist to construct the states. alist is stored as a list of lists and the indices are $a[timestep][gate]$. For a timestep $\Delta \tau$, the gate indices of $a$ correspond to:

$$
a[0]=e^{-ia[II]\Delta\tau\hat{I} \otimes \hat{I}} \text{,  } \; 
a[1]=e^{-ia[IX]\Delta\tau\hat{I} \otimes \hat{X}} \text{,  } \;
a[2]=e^{-ia[IY]\Delta\tau\hat{I} \otimes \hat{Y}} \text{,  } \;
a[3]=e^{-ia[ZZ]\Delta\tau\hat{I} \otimes \hat{Z}}
$$
$$
a[4]=e^{-ia[XI]\Delta\tau\hat{X} \otimes \hat{I}} \text{,  } \; 
a[5]=e^{-ia[XX]\Delta\tau\hat{X} \otimes \hat{X}} \text{,  } \;
a[6]=e^{-ia[XY]\Delta\tau\hat{X} \otimes \hat{Y}} \text{,  } \;
a[7]=e^{-ia[XZ]\Delta\tau\hat{X} \otimes \hat{Z}}
$$
$$
a[8]=e^{-ia[YI]\Delta\tau\hat{Y} \otimes \hat{I}} \text{,  } \; 
a[9]=e^{-ia[YX]\Delta\tau\hat{Y} \otimes \hat{X}} \text{,  } \;
a[10]=e^{-ia[YY]\Delta\tau\hat{Y} \otimes \hat{Y}} \text{,  } \;
a[11]=e^{-ia[YZ]\Delta\tau\hat{Y} \otimes \hat{Z}}
$$
$$
a[12]=e^{-ia[ZI]\Delta\tau\hat{Z} \otimes \hat{I}} \text{,  } \; 
a[13]=e^{-ia[ZX]\Delta\tau\hat{Z} \otimes \hat{X}} \text{,  } \;
a[14]=e^{-ia[ZY]\Delta\tau\hat{Z} \otimes \hat{Y}} \text{,  } \;
a[15]=e^{-ia[ZZ]\Delta\tau\hat{Z} \otimes \hat{Z}}
$$

The 0 index stores the coefficient for the identity matrix $\hat{I}$ on both qubits, which is a global phase for each qubit that we can ignore.

We can break down the matrix exponential of the Kronecker product of two Pauli matrices as:

$$e^{-i \theta \Delta\tau \bigotimes_{j} \sigma_{j}} = \cosh(-i \theta \Delta\tau) \bigotimes_{j} I + \sinh(-i \theta \Delta\tau) \bigotimes_{j} \sigma_{j} = \cos(\theta \Delta\tau) \bigotimes_{j} I - i \sin(\theta \Delta\tau) \bigotimes_{j} \sigma_{j}$$

For example,

$$e^{-ia[XY]\Delta\tau\hat{X} \otimes \hat{Y}} = \cos(a[XY] \Delta\tau) (I \otimes I) - i \sin(a[XY] \Delta\tau) (X \otimes Y)$$

To implement this in terms of gates on a quantum computer, refer to page 210 in Nielsen and Chuang: http://mmrc.amss.cas.cn/tlb/201702/W020170224608149940643.pdf

We need to apply a phase shift to the system. Phase is $e^{-i \Delta \tau}$ if the parity of the $n$ qubits in the computational basis is even, and $e^{i \Delta \tau}$ if odd. For 2 qubits, this is the matrix exponential of $i \Delta \tau Z \otimes Z$

$$ e^{-i \Delta \tau Z \otimes Z} =
\begin{pmatrix}
e^{-i \Delta \tau} & 0 & 0 & 0\\
0 & e^{i \Delta \tau} & 0 & 0\\
0 & 0 & e^{i \Delta \tau} & 0\\
0 & 0 & 0 & e^{-i \Delta \tau}\\
\end{pmatrix}
$$

The function **applyPhase** implements this $e^{-i \Delta \tau Z \otimes Z}$ operation

In [5]:
def applyPhase(qc,qbits,angle, id_idx_list):
    a = list(range(0,len(qbits)-1))
    b = id_idx_list
    c = list(set(a).difference(set(b)))
    for i in c:
        qc.cx(qbits[i],qbits[-1])
    qc.rz(angle,qbits[-1])
    c.reverse()
    for i in c:
        qc.cx(qbits[i],qbits[-1])

# def applyPhase(qc,qbits,angle):
#     c = list(range(0,len(qbits)-1))
#     for i in c:
#         qc.cx(qbits[i],qbits[-1])
#     qc.rz(angle,qbits[-1])
#     for i in c:
#         qc.cx(qbits[i],qbits[-1])

In [6]:
def propagate(qc,qbits,alist):
    nqbits = len(qbits)
    # Circuit to propagate the state
    if len(alist) == 0:
        None
    else:
        for t in range(len(alist)):
            for i in range(1,4**nqbits):
                
                angle = np.real(alist[t][i])
                
                idx_pair = pauli_pairs[i]
                
                # single qubit rotation case... all qubits acted on by I except for one
                nonzero_count = 0
                single_rot_q = -1
                for j in range(len(idx_pair)):
                    if idx_pair[j] != 0:
                        nonzero_count += 1
                        single_rot_q = j
                    if nonzero_count > 1:
                        single_rot_q = -1
                        break   
                
                if single_rot_q != -1:
                    if idx_pair[single_rot_q] == 1:
                        qc.rx(angle,qbits[single_rot_q])
                    elif idx_pair[single_rot_q] == 2:
                        qc.ry(angle,qbits[single_rot_q])
                    elif idx_pair[single_rot_q] == 3:
                        qc.rz(angle,qbits[single_rot_q])
                
                # main case...
                else:
                    id_idx_list = []
                    for j in range(len(idx_pair)):
                        if idx_pair[j] == 0:
                            id_idx_list.append(j)
#                               pass
                        elif idx_pair[j] == 1:
                            qc.h(qbits[j])
                        elif idx_pair[j] == 2:
                            qc.rx(np.pi/2, qbits[j])
                        elif idx_pair[j] == 3:
                            pass
                        else:
                            raise ValueError
                            
                    # maybe skip the CNOT on qubits acted on by I?
                    applyPhase(qc,qbits,angle, id_idx_list)

#                     applyPhase(qc,qbits,angle)

                    # apply conjugate transpose
                    for j in range(len(idx_pair)):
                        if idx_pair[j] == 0:
                            pass
                        elif idx_pair[j] == 1:
                            qc.h(qbits[j])
                        elif idx_pair[j] == 2:
                            qc.rx(-np.pi/2, qbits[j])
                        elif idx_pair[j] == 3:
                            pass
                        else:
                            raise ValueError

## Update Rotation Angles For Unitary Imaginary Time Evolution

We denote **alist** as $a[m]$, the matrix of rotations per imaginary time step for our unitary operator that recreates imaginary time evolution. Recall:

$$ A[m] = \sum_{i_{1} \ldots i_{k}} a[m]_{i_{1} \ldots i_{k}} \sigma_{i_{1}} \ldots \sigma_{i_{k}} \equiv \sum_{I} a[m]_{I} \sigma_{I}$$

The idea behind QITE is to define a unitary operator $e^{-i \Delta \tau A[m]}$ and apply it to a state $| \Psi \rangle$ to reproduce the state:

$$| \Psi' \rangle = c^{-1/2} e^{- \Delta \tau h[m]} | \Psi \rangle$$

We define the distance between the desired and initial states as $| \Delta_{0} \rangle = \left( {| \Psi' \rangle - | \Psi \rangle} \right) /{\Delta \tau}$ and the difference between the evolved and initial states as $| \Delta \rangle = -iA[m] | \Psi \rangle$. The goal is to minimize $\| \Delta_{0} - \Delta \|$. This corresponds to minimizing the quadratic function (from SI of the Motta paper):

$$f \left( a[m] \right) = f_{0} + \sum_{I} b_{I} a[m]_{I} + \sum_{I,J} a[m]_{I} S_{IJ} a[m]_{J}$$
$$f_{0} = \langle \Delta_{0} | \Delta_{0} \rangle \text{,  } S_{IJ} = \langle \Psi | \sigma_{I}^{\dagger} \sigma_{J} | \Psi \rangle \text{,  } b_{I} = i \langle \Psi | \sigma_{I}^{\dagger} | \Delta_{0} \rangle - i \langle \Delta_{0} | \sigma_{I} | \Psi \rangle = -\frac{i}{\sqrt{c}} \langle \Psi | \sigma_{I}^{\dagger} h[m] | \Psi \rangle$$

The minimum is the solution to the linear equation $\left( S + S^{T} \right) a[m] = -b$, which we solve by applying the generalized inverse or via some iterative algorithm.

To compute $S$, we need to the matrix elements:

$$S_{ij} = \langle \psi | Q_{i} Q_{j} | \psi \rangle = \langle \psi | ( \sigma_{i0} \otimes \sigma_{i1} ) ( \sigma_{j0} \otimes \sigma_{j1} ) | \psi \rangle = \langle \psi | \left( \sigma_{i0} \sigma_{j0} \otimes \sigma_{i1} \sigma_{j1} \right) | \psi \rangle$$

To compute $b$, we need the vector elements:
$$b_{i} = i \langle \psi | Q_{i} | \Delta_{0} \rangle - h.c. = i \langle \psi | {( \sigma_{i0} \otimes \sigma_{i1})} | \Delta_{0} \rangle - h.c.$$


These computations are done in the **update_alist** function. But to compute these elements, we first need to compute the 2-qubit Pauli expectations in a state propagated by our unitary imaginary time evolution.

For the 1-qubit case, we need to construct the matrix $S_{ij} = \langle \psi | \sigma_{i} \sigma_{j} | \psi \rangle$ and the vector $b_{i}$. Well, our earlier functions allow us to measure the expectation values of the different pauli matrices. How do we obtain $S_{ij}$ from a list of $\langle \psi | \sigma | \psi \rangle$? We can exploit the fact that up to some coefficients, $\sigma_{i}\sigma_{j} = c_{ij}\sigma_{ij}$. For example, $\sigma_{x}\sigma_{y} = i\sigma_{z}$. We will need a matrix to keep track of what pauli matrix and coefficient we get for $\sigma_{i}\sigma_{j}$. A pragmatic approach is to hard-code these 1-qubit matrix multiplication matrices.

For the 2-qubit case, we can reuse the 1-qubit matrices. For example,
$(X \otimes X) (X \otimes Y) = XX \otimes XY = I \otimes i Z$

Another way is to hard-code the 2-qubit matrices by directly populating the Lie algebra rules, as we do below.

In [7]:
import sys
sys.path.append('classical')

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm 
from binary_functions import Int2Bas,Bas2Int,Opp2Str,Str2Opp
from Pmn import PPmunu

In [9]:
def lie_algebra(mu,nu,n):
    # Return coefficients and index for sigma mu,sigma nu
    index = ''
    coeff = 1
    for i in range(n):
        tmpA,tmpB = PPmunu(mu[i]+nu[i])
        index += tmpA
        coeff *= tmpB
    return coeff,Bas2Int(Str2Opp(index),4)

In [10]:
def get_index_coeff(n=2):
    index = np.zeros([4**n,4**n],dtype=int)
    coeff = np.zeros([4**n,4**n],dtype=complex)
    for i in range(4**n):
        for j in range(4**n):
            coeff[i,j],index[i,j] = lie_algebra(Opp2Str(Int2Bas(i,4,n)),Opp2Str(Int2Bas(j,4,n)),n)
    return index,coeff

In [11]:
index, coeff = get_index_coeff(n=4)
# index, coeff = get_index_coeff(n=3)
# index, coeff = get_index_coeff(n=2)

print("index matrix: ")
print(index)
print("coeff matrix: ")
print(coeff)

index matrix: 
[[  0   1   2 ... 253 254 255]
 [  1   0   3 ... 252 255 254]
 [  2   3   0 ... 255 252 253]
 ...
 [253 252 255 ...   0   3   2]
 [254 255 252 ...   3   0   1]
 [255 254 253 ...   2   1   0]]
coeff matrix: 
[[1.+0.j 1.+0.j 1.+0.j ... 1.+0.j 1.+0.j 1.+0.j]
 [1.+0.j 1.+0.j 0.+1.j ... 1.+0.j 0.+1.j 0.-1.j]
 [1.+0.j 0.-1.j 1.+0.j ... 0.-1.j 1.+0.j 0.+1.j]
 ...
 [1.+0.j 1.+0.j 0.+1.j ... 1.+0.j 0.+1.j 0.-1.j]
 [1.+0.j 0.-1.j 1.+0.j ... 0.-1.j 1.+0.j 0.+1.j]
 [1.+0.j 0.+1.j 0.-1.j ... 0.+1.j 0.-1.j 1.+0.j]]


We now want to obtain the coefficients $a[m]$ at the current time step and append to alist. In the **update_alist** function, we use the **index** and **coeff** matrices and the function np.linalg.std to construct the S matrix and b vector and solve for x, the rotation angle used in state propagation. Because the rotation gate is defined as $e^{-i\theta/2\sigma}$, we multiply the x by 2 before storing them in alist. More specifically:

In [12]:
def update_alist(sigma_expectation,alist,db,delta,hm):
    '''
    Obtain A[m]
    To do this, we compute the S matrix and the b vector. We also need to compute the norm c
    Details:
    Each local Hamiltonian term hm can be a sum of many 2-qubit operators...
     ... see the construction of the hm_list to see why... for each term in hm (hm[i]):
    hm[i][1][0] is the multiplicative constant in the Hamiltonian
    hm[i][0][0] is the index of the Pauli matrix for qubit 0
    hm[i][0][1] is the index of the Pauli matrix for qubit 1
    '''
    
    # c is the squared norm
    c = 1
    for i in range(len(hm)):
        hm_idx = pauli_pair_dict[tuple(hm[i][0])]
        c -= 2 * db * hm[i][1][0] * sigma_expectation[hm_idx]
    # c is now the norm, after we take its square root
    c = np.sqrt(c)
    
    dim = len(sigma_expectation)
    
    # Initialize S matrix
    S = np.zeros([dim, dim], dtype=complex)
    # Initialize b vector
    b = np.zeros([dim], dtype=complex)
    
    for i in range(dim):
        # Step 1: Obtain S matrix
        for j in range(dim):
            S[i, j] = sigma_expectation[index[i,j]] * coeff[i,j]
#             print(i, j, S[i, j])

        # Step 2: Obtain b vector
        b[i] += (sigma_expectation[i]/c-sigma_expectation[i])/(db)
        # iterate through hm terms
        for j in range(len(hm)):
            hm_idx = pauli_pair_dict[tuple(hm[j][0])]            
            b[i] -= hm[j][1][0]*coeff[i,hm_idx]*sigma_expectation[index[i,hm_idx]]/c
        b[i] = 1j*b[i] - 1j*np.conj(b[i])
#         print(i, b[i])
        
    # Step 3: Add regularizer... dim x dim matrix with delta on main diagonal and 0s elsewhere
    dalpha = np.eye(dim)*delta

    # Step 4: Solve for linear equation, the solution is multiplied by -2 because of the definition of unitary rotation gates is exp(-i theta/2)
    x = np.linalg.lstsq(S + np.transpose(S) + dalpha, -b, rcond=-1)[0]
    alist.append([])
    for i in range(len(x)):
        alist[-1].append(-x[i] * 2 * db)
    return c



## Full QITE Protocol

Now we implement the full QITE protocol. It will be good to have a big picture of what should be done. We step through imaginary time and at each time step, we should first measure the expectation values of the pauli matrices $\sigma$. This is indicated in the first two lines of the for loop. Using the relevant expectation values, we obtain the coefficients $a[m]$ in equation (2.5) of Lecture 12 using the update rule in equation (2.9-2.11). We store it in a list and use this to propagate our state. Note we have to always reconstruct our state for each new measurements we make. Finally, we measure the current energy values. Let us construct the required functions starting with **get_expectation**. We make use of the **measure** function we constructed earlier. The key here is to propagate our state using the coefficients in alist. We do this using the **propagate** function.

In [13]:
def ansatz(p, qbits):
    None

def measure_energy(qbits, alist, shots, hm_list, display=False):
    # Measure the energy at the end of each time step
    Energy = 0
    Nterms = len(hm_list)
    nqbits = len(qbits)
    for i in range(len(hm_list)):
        hm = hm_list[i]
        for j in range(len(hm)):
                        
            pauli_pair = tuple(hm[j][0])
            idx = pauli_pair_dict[pauli_pair]
            
            pauli_expectation = 1
            
            # I x I ....
            cond1 = [0] * nqbits
            # Z x I ....
            cond2 = [3] + [0] * (nqbits - 1)
    
            if pauli_pair != tuple(cond1) and pauli_pair != tuple(cond2):
                # quantum nqbits register
                qr = QuantumRegister(len(qbits))
                # classical 1 bit readout register
                ro = ClassicalRegister(1, name='ro')
                # our combined circuit
                qc = QuantumCircuit(qr, ro)

                propagate(qc,qbits,alist)

                measure(qc,ro,qbits,idx)

                # Execute
                job = execute(qc, backend, shots=shots, optimization_level=1)
                counts = job.result().get_counts()
                if '0' not in counts:
                    counts['0'] = 0
                if '1' not in counts:
                    counts['1'] = 0
                pauli_expectation = (counts['0'] - counts['1']) / shots
            
            tmp = hm[j][1][0]*pauli_expectation
            print("pauli_pair {} energy contribution: {}".format(pauli_pair, tmp))
            Energy += tmp
        if display:
        # qc after last Pauli string measurement
            print(qc)
    return Energy

# use this to avoid the extra time complexity of propagating and measuring again... the values we want are already in sigma_expectation
def get_energy_from_sigma(sigma_expectation, hm_list, verbose=True):
    Energy = 0
    Nterms = len(hm_list)
    for i in range(len(hm_list)):
        hm = hm_list[i]
        # For each Pauli matrix pair (2-qubit operator) in the Hamiltonian, 
        for j in range(len(hm)):
            # pauli pair to retrieve the measurement for in sigma_expectations
            pauli_pair = tuple(hm[j][0])
            # energy contribution of this term
            tmp = hm[j][1][0] * sigma_expectation[pauli_pair_dict[pauli_pair]]
            if verbose:
                print("pauli_pair {} energy contribution: {}".format(pauli_pair, tmp))
            Energy += tmp
    return Energy

def get_expectation(qbits, alist, shots, display=False):
    # Obtain the expectation values of the Pauli string at each time step
    dim = 4**len(qbits)
    nqbits = len(qbits)
    sigma_expectation = np.zeros([dim], dtype=complex)
    for i in range(dim): 
        pauli_expectation = 1
        
        # I x I ....
        cond1 = [0] * nqbits
        # Z x I ....
        cond2 = [3] + [0] * (nqbits - 1)

        if pauli_pairs[i] != tuple(cond1) and pauli_pairs[i] != tuple(cond2):
            # quantum nqbits register
            qr = QuantumRegister(len(qbits))
            # classical 1 bit readout register
            ro = ClassicalRegister(1, name='ro')
            # our combined circuit
            qc = QuantumCircuit(qr, ro)

            propagate(qc,qbits,alist)

            measure(qc,ro,qbits,i)

            # Execute
            job = execute(qc, backend, shots=shots, optimization_level=1)
            counts = job.result().get_counts()
            if '0' not in counts:
                counts['0'] = 0
            if '1' not in counts:
                counts['1'] = 0
            pauli_expectation = (counts['0'] - counts['1']) / shots
        
        sigma_expectation[i] = pauli_expectation
        
    if display:
        print(qc)
    return sigma_expectation

def qite_step(qbits, alist, shots, db, delta, hm_list, verbose=True, display=False):
    sigma_expectation = get_expectation(qbits, alist, shots, display=display)
    energy = get_energy_from_sigma(sigma_expectation, hm_list, verbose=verbose)
    for j in range(len(hm_list)):
        update_alist(sigma_expectation, alist, db, delta, hm_list[j]) # = norm
    return alist, sigma_expectation, energy

def qite(qbits, shots, db, delta, N, hm_list, verbose=True, display=False):
    E = np.zeros([N+1],dtype=complex)
    alist = []

    # Qite main loop
    QITE_expectations = []
    for i in range(0,N):
        alist, sigma_expectation, energy = qite_step(qbits, alist, shots, db, delta, hm_list, verbose=verbose, display=display)
        QITE_expectations.append(sigma_expectation.real.tolist())
        E[i] = energy
        
        if verbose:
            if i == 0:
                print("Initial Energy: ", E[0])
            else: 
                # print("QITE Step: ", i)
                print("Imaginary Time: ", float(i) * db)
                print("Energy: ", E[i])
#             print("Pauli Expectations: ", sigma_expectation.real.tolist())
    
    # measure final energy
    E[N] = measure_energy(qbits, alist, shots, hm_list, display=display)
    
    print("Imaginary Time: ", N * db)
    print("Final energy: ", E[N])
    
    return E, QITE_expectations

## Running QITE with 2-site 1D Hubbard Hamiltonian

The 2-site 1D Hubbard Hamiltonian, for a half-filled lattice (2 fermions) is:

$$ H = -t \sum_{\sigma} \left( a_{1 \sigma}^{\dagger} a_{2 \sigma} + a_{2 \sigma}^{\dagger} a_{1 \sigma} \right) + U \sum_{i=1}^{2} n_{i \uparrow} n_{i \downarrow} $$

We can solve for the ground state energy of this Hamiltonian exactly, since the Hilbert space is small enough. We expect the ground state wavefunction to be symmetric with spin up and spin down components, and the most general guess is:

$$| \psi \rangle = \alpha \left( a_{1 \uparrow}^{\dagger} a_{1 \downarrow}^{\dagger} + a_{2 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} \right) |0 \rangle + \beta \left( a_{1 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} + a_{1 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} \right) |0 \rangle$$

Using the time independent Schrodinger equation $H | \psi \rangle = E | \psi \rangle$ we get the coupled equations:

$$ (E-U) \alpha + 2 t \beta = 0$$
$$2t \alpha + E \beta = 0$$

There are two solutions; the one with the lower energy is the ground state energy:

$$E_{0} = \frac{1}{2} \left( U - \sqrt{U^{2} + 16 t^{2}} \right)$$

For our simulation, we run QITE with $t=1$ and $U=2$, so our goal is to match the exact ground state energy of $E_{0} = -1.23607$

We run QITE with a two qubit QVM using the 2-site 1D Hubbard Hamiltonian, with fermionic operators mapped to Pauli matrices by the Jordan-Wigner transformation.

$$H = -t \left( X \otimes I + I \otimes X \right) + \frac{U}{2} \left( I + Z \otimes Z \right)$$

In [14]:
def run_QITE(hm_list, nqbits=2, N=10, shots=8192, delta=0.25, db=0.1, verbose=True, display=False):    
    # enumerate the qbits... makes it easier to be dynamic if we have a lot of them
    qbits = [i for i in range(0,nqbits)]
#     qbits.reverse()
    
    print("qubit ordering: ", qbits)
    
    print("hm list: ", hm_list)
    print("running QITE...")
    E, QITE_expectations = qite(qbits,shots,db,delta,N,hm_list,verbose=verbose,display=display)
    return E, QITE_expectations

In [15]:
# it seems like best delta is between 0.26, 0.28, 0.30... in this neighborhood
nqbits = 4
# nqbits = 3
# nqbits = 2

N = 20
db = 0.05
delta = 0.30
shots = 16384

In [16]:
# kinetic energy contribution
t = 1.
# potential energy contribution
U = 2.
print("t = ", t, ", U = ", U)

# # 2 qubit
# hm_list = []
# # the first term (hopping kinetic term) of H
# hm_list.append([])
# hm_list[0].append([[1, 0], [-t]])
# hm_list[0].append([[0, 1], [-t]])
# hm_list[0].append([[0, 0], [U/2]])
# hm_list[0].append([[3, 3], [U/2]])

# # 3 qubit
# hm_list = []
# # the first term (hopping kinetic term) of H
# hm_list.append([])
# hm_list[0].append([[1, 0, 0], [-t]])
# hm_list[0].append([[0, 1, 0], [-t]])
# hm_list[0].append([[0, 0, 0], [U/2]])
# hm_list[0].append([[3, 3, 0], [U/2]])

# 4 qubit
hm_list = []
# the first term (hopping kinetic term) of H
hm_list.append([])
hm_list[0].append([[1, 0, 0, 0], [-t]])
hm_list[0].append([[0, 1, 0, 0], [-t]])
hm_list[0].append([[0, 0, 0, 0], [U/2]])
hm_list[0].append([[3, 3, 0, 0], [U/2]])

t =  1.0 , U =  2.0


In [17]:
# Compute exact ground state energy, 4 qubits version of Hamiltonian
t_hubbard = 1
U_hubbard = 2
sigma = np.zeros([2, 2, 4], dtype=complex)
sigma[0, 0, 0] = 1.
sigma[1, 1, 0] = 1.
sigma[0, 1, 1] = 1.
sigma[1, 0, 1] = 1.
sigma[0, 1, 2] = -1j
sigma[1, 0, 2] = 1j
sigma[0, 0, 3] = 1.
sigma[1, 1, 3] = -1.

a = np.kron(sigma[:, :, 1], sigma[:, :, 0])
b = np.kron(sigma[:, :, 0], sigma[:, :, 0])
c = np.kron(sigma[:, :, 0], sigma[:, :, 1])
d = np.kron(sigma[:, :, 3], sigma[:, :, 3])

Ham = -1 * (np.kron(a,b) + np.kron(c,b)) + (np.kron(b,b) + np.kron(d,b))
w, v = np.linalg.eigh(Ham)
print('The exact solution is ', w[0])

The exact solution is  -1.2360679774997902


In [18]:
# E, QITE_expectations = run_QITE(hm_list, nqbits=nqbits, N=N, db=db, delta=delta, shots=shots, display=False)

In [19]:
E_list = []
n_trials = 1
verbose = False
if n_trials == 1:
    verbose = True
for i in range(n_trials):
    print("Trial ", i+1)
    E, _ = run_QITE(hm_list, nqbits=nqbits, N=N, db=db, delta=delta, shots=shots, verbose=verbose, display=False)
    E_list.append(E.real)

Trial  1
qubit ordering:  [0, 1, 2, 3]
hm list:  [[[[1, 0, 0, 0], [-1.0]], [[0, 1, 0, 0], [-1.0]], [[0, 0, 0, 0], [1.0]], [[3, 3, 0, 0], [1.0]]]]
running QITE...
pauli_pair (1, 0, 0, 0) energy contribution: (0.0023193359375-0j)
pauli_pair (0, 1, 0, 0) energy contribution: (0.004638671875-0j)
pauli_pair (0, 0, 0, 0) energy contribution: (1+0j)
pauli_pair (3, 3, 0, 0) energy contribution: (1+0j)
Initial Energy:  (2.0069580078125+0j)
pauli_pair (1, 0, 0, 0) energy contribution: (-0.122314453125+0j)
pauli_pair (0, 1, 0, 0) energy contribution: (-0.118896484375+0j)
pauli_pair (0, 0, 0, 0) energy contribution: (1+0j)
pauli_pair (3, 3, 0, 0) energy contribution: (0.9866943359375+0j)
Imaginary Time:  0.05
Energy:  (1.7454833984375+0j)
pauli_pair (1, 0, 0, 0) energy contribution: (-0.2122802734375+0j)
pauli_pair (0, 1, 0, 0) energy contribution: (-0.218017578125+0j)
pauli_pair (0, 0, 0, 0) energy contribution: (1+0j)
pauli_pair (3, 3, 0, 0) energy contribution: (0.945068359375+0j)
Imaginary Tim

Process ForkProcess-2:
Process ForkProcess-3:
Process ForkProcess-4:
Process ForkProcess-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/abao/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abao/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/abao/anaconda3/lib/python3.7/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/abao/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/abao/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/abao/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abao/anaconda3/lib/python3.7/multiprocess

KeyboardInterrupt: 

In [ ]:
def plot_E_list(E_list):
    plt.figure(figsize=[8,4], dpi=150)

    if len(E_list) > 1:
        for i in range(len(E_list)):
            E = E_list[i]
            plt.plot(np.arange(0,N+1)*db,E.real,'-r',alpha=0.2)

        plt.plot(np.arange(0,N+1)*db,np.average(E_list,axis=0),'-ro', label="QITE")

    else:
        plt.plot(np.arange(0,N+1)*db,E_list[0],'-ro', label="QITE")

    plt.axhline(y=w[0], color='k',linestyle='--',label="Ground state")
    plt.title("QITE on {} Qubits".format(nqbits))
    plt.xlabel("Imaginary time")
    plt.ylabel("Energy")
    plt.grid()
    plt.legend(bbox_to_anchor=(1.0,1.0))
    plt.show()

In [ ]:
plot_E_list(E_list)